In [ ]:
import pandas as pd
import numpy as np
import datetime
import socket
import struct
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
headerString = "date time time-taken c-ip cs-username cs-auth-group x-exception-id sc-filter-result cs-categories cs(Referer) sc-status s-action cs-method rs(Content-Type) cs-uri-scheme cs-host cs-uri-port cs-uri-path cs-uri-query cs-uri-extension cs(User-Agent) s-ip sc-bytes cs-bytes x-virus-id"
header=headerString.split(" ")
a = pd.read_csv('./Data/message.txt', delimiter="\s+", index_col=False, encoding="utf-8", comment='#',names=header)


C:\Users\VRUma\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10,16,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#drop columns : cs-unsername cs-auth-group  (date prob)
df1 = a.sample(frac=0.5,random_state=200) 
df2 =df1.sample(frac=0.5,random_state=200)

df1=df1.drop(columns=['cs-username','cs-auth-group','date',
                      'time-taken','cs-method',
                      'rs(Content-Type)','cs-uri-path',
                      'cs-uri-query','cs-uri-extension','cs(User-Agent)','s-ip','x-virus-id'])


In [4]:

def preprocessing(data):
    data=data.dropna()
    data.loc[data['x-exception-id'] == '-', 'x-exception-id'] = ""

    return data


In [5]:
def compute_similarity(string1,stand_val):
    indexes=np.where(string1==stand_val)
    return indexes[0]


def apply_sim(data,column_name,one_line_df,new_col_name):
    if column_name not in data.columns:
        print("Wrong column name")
        return 0
    else :
        list_indexes=compute_similarity(data[column_name].values,one_line_df[column_name].values[0])
        data[new_col_name]=data.index.isin(list_indexes)
        data[new_col_name]=data[new_col_name].astype(int)
    return data


In [6]:
def compute_all_sims(data,y):
    all_cols=['x-exception-id','cs-categories',
 'sc-status','s-action','cs-uri-scheme','cs-uri-port']
    all_cols
    for col in all_cols:
        apply_sim(data,col,y,'sim')
        if 'final_sim' not in data.columns :
            data['final_sim'] = data['sim']
        else :
            data['final_sim']= data['final_sim'] + data['sim']
    return data


In [7]:
def add_urlsize(data):
    data["url_size"] = data["cs(Referer)"].apply(lambda x: len(str(x)))
    return data

def add_hostfreq(data):
    cs_host_frequency = data.groupby(['cs-host']).count().sort_values(by=["url_size"],ascending=False)
    most_frequent = cs_host_frequency.index.tolist()
    quantity = cs_host_frequency['s-action'].tolist()
    frequent_host_dict = dict(zip(most_frequent, quantity))
    data["cs-host-frequency"] = data["cs-host"].apply(lambda x: frequent_host_dict.get(x))
    return data






In [ ]:
def normalization(data):
    df4=data.copy()
    df4['final_sim']=(df4['final_sim']-df4['final_sim'].mean())/df4['final_sim'].std()
    #df4 = df4.drop([283551], axis=0)
    df4['sc-bytes'] = pd.to_numeric(df4['sc-bytes'])
    df4['cs-bytes'] = pd.to_numeric(df4['cs-bytes'])
    df4['sc-bytes']=(df4['sc-bytes']-df4['sc-bytes'].mean())/df4['sc-bytes'].std()
    df4['cs-bytes']=(df4['cs-bytes']-df4['cs-bytes'].mean())/df4['cs-bytes'].std()
    df4['url_size'] =(df4['url_size']-df4['url_size'].mean())/df4['url_size'].std()
    df4['cs-host-frequency'] = (df4['cs-host-frequency']-df4['cs-host-frequency'].mean())/df4['cs-host-frequency'].std()
    return df4


def normalization2(data):
    df4=data.copy()
    df4['final_sim']=(df4['final_sim']-df4['final_sim'].min())/(df4['final_sim'].max() -df4['final_sim'].min()) 
    #df4 = df4.drop([283551], axis=0)
    df4['sc-bytes'] = pd.to_numeric(df4['sc-bytes'])
    df4['cs-bytes'] = pd.to_numeric(df4['cs-bytes'])
    df4['sc-bytes']=(df4['sc-bytes']-df4['sc-bytes'].min())/(df4['sc-bytes'].max() -df4['sc-bytes'].min()) 
    df4['cs-bytes']=(df4['cs-bytes']-df4['cs-bytes'].min())/(df4['cs-bytes'].max() -df4['cs-bytes'].min()) 
    return df4




def make_dummies(data, string):
    data= pd.concat([data,pd.get_dummies(data[string], prefix=string)],axis=1)
    data.drop([string], axis=1, inplace=True)
    return data



In [ ]:

y=df1.tail(1)
y
df3=df1.sample(frac=0.4,random_state=2000)
df3=df3.reset_index(drop=True)
df3=preprocessing(df3)
df3=make_dummies(df3,'sc-filter-result')
df3

In [ ]:
compute_all_sims(df3,y)
df3=add_urlsize(df3)
df3=add_hostfreq(df3)
dfn=normalization(df3)
dfn1=dfn.sample(frac=0.5,random_state=1337)

In [ ]:
from sklearn.cluster import DBSCAN

#dfn=dfn.dropna()
clusters = DBSCAN(eps = 0.7, min_samples=100)
clusters.fit(dfn1[['cs-bytes','sc-bytes','url_size','cs-host-frequency','sc-filter-result_DENIED','sc-filter-result_OBSERVED','sc-filter-result_PROXIED']])
dfn1['labels'] = [str(e) for e in clusters.labels_]

dfn1

In [ ]:
dfn1.groupby('labels').count()

In [ ]:
dfn1.loc[dfn1['labels']=="-1"]

In [ ]:

dfn2=dfn1.loc[dfn1['cs-bytes']<1]
plt.scatter(dfn2['final_sim'],dfn2['cs-bytes'])

In [ ]:
dfn2=dfn2.loc[dfn2['sc-bytes']<1]

plt.scatter(dfn2['final_sim'],dfn2['sc-bytes'])